In [1]:
import os

In [2]:
pwd

'c:\\Users\\ten\\Desktop\\Laptop_Price\\research\\testing'

In [3]:
os.chdir("../../")

In [4]:
pwd

'c:\\Users\\ten\\Desktop\\Laptop_Price'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    model_name: str
    algorithm: str
    n_neighbors: float
    weights: str

In [6]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            model_name = config.model_name,
            algorithm = params.algorithm,
            n_neighbors = params.n_neighbors,
            weights = schema.weights
            
        )

        return model_trainer_config

In [9]:
import pandas as pd
import os
from mlProject import logger
from sklearn.neighbors import KNeighborsRegressor
import joblib

In [10]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    
    def train(self,x_train,x_test,y_train,y_test):
        
        knn_final = KNeighborsRegressor(algorithm=self.config.algorithm,n_neighbors=self.config.n_neighbors,weights=self.config.weights)
        knn_final.fit(x_train,y_train)

        joblib.dump(knn_final, os.path.join(self.config.root_dir, self.config.model_name))

